 # Opis notatnika
 W tym notatniku tworzymy dedykowaną bazę danych wraz ze strukturą tabel, która zostanie uzupełniona w nastęnym notatniku.

##### Importujemy potrzebne biblioteki

In [26]:
import psycopg2
from dotenv import load_dotenv
import os

 ## Połączenie z bazą danych

In [29]:
# Ładujemy zmienne środowiskowe z pliku .env
load_dotenv('DB_pass.env')

True

In [38]:
# Używamy zmiennych środowiskowych do połączenia się z istniejącą bazą danych
connection = psycopg2.connect(
    dbname=os.getenv("O_DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

connection.autocommit = True
cursor = connection.cursor()

# Tworzymy nową bazę danych
cursor.execute("CREATE DATABASE airlines_project;")
conn.close()

In [92]:
# Używamy zmiennych środowiskowych do połączenia się z nowo utworzoną bazą danych
connection = psycopg2.connect(
    dbname=os.getenv("N_DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

cursor = connection.cursor()

 ## Wczytanie pliku `database_schema.sql`
 Z katalogu `sql` wczytujemy plik `database_schema.sql`

In [42]:
# Wczytujemy strukturę bazy danych z pliku .sql
with open('../sql/database_schema.sql', 'r') as file:
    schema_sql = file.read()

In [48]:
print(schema_sql)

DROP TABLE IF EXISTS public.aircraft CASCADE;
CREATE TABLE IF NOT EXISTS public.aircraft
(
    id serial,
    manufacture_year bigint,
    tail_num text,
    number_of_seats double precision
)
;

CREATE OR REPLACE VIEW public.v_aircraft_blocker AS
SELECT 1 FROM public.aircraft LIMIT 1;

DROP TABLE IF EXISTS public.airport_weather CASCADE;
CREATE TABLE IF NOT EXISTS public.airport_weather
(
    id serial,
    station text,
    name text,
    date text,
    awnd double precision,
    pgtm double precision,
    prcp double precision,
    snow double precision,
    snwd double precision,
    tavg double precision,
    tmax double precision,
    tmin double precision,
    wdf2 double precision,
    wdf5 double precision,
    wsf2 double precision,
    wsf5 double precision,
    wt01 double precision,
    wt02 double precision,
    wt03 double precision,
    wt04 double precision,
    wt05 double precision,
    wt06 double precision,
    wt07 double precision,
    wt08 double precision,
    

In [139]:
# Dzielimy zawartość pliku na poszczególne komendy zakończone średnikiem
sql_commands = schema_sql.split(';')

In [143]:
# Dodajemy średnik na końcu każdej komendy, czyścimy tekst z niepotrzbnych znaków białych
sql_commands = [command.strip() + ';' for command in sql_commands]

In [145]:
sql_commands

['DROP TABLE IF EXISTS public.aircraft CASCADE;',
 'CREATE TABLE IF NOT EXISTS public.aircraft\n(\n    id serial,\n    manufacture_year bigint,\n    tail_num text,\n    number_of_seats double precision\n);',
 'CREATE OR REPLACE VIEW public.v_aircraft_blocker AS\nSELECT 1 FROM public.aircraft LIMIT 1;',
 'DROP TABLE IF EXISTS public.airport_weather CASCADE;',
 'CREATE TABLE IF NOT EXISTS public.airport_weather\n(\n    id serial,\n    station text,\n    name text,\n    date text,\n    awnd double precision,\n    pgtm double precision,\n    prcp double precision,\n    snow double precision,\n    snwd double precision,\n    tavg double precision,\n    tmax double precision,\n    tmin double precision,\n    wdf2 double precision,\n    wdf5 double precision,\n    wsf2 double precision,\n    wsf5 double precision,\n    wt01 double precision,\n    wt02 double precision,\n    wt03 double precision,\n    wt04 double precision,\n    wt05 double precision,\n    wt06 double precision,\n    wt07 dou

##### W tym miejscu wczytujemy każdą z kwerend, aby zainicjować strukturę bazy danych

In [94]:
try:
    # Iterujemy przez każdą komendę SQL
    for command in sql_commands:
        cursor.execute(command)

    # Jeśli wszystkie komendy zakończą się sukcesem, zatwierdzamy transakcję
    connection.commit()
    print("Wszystkie operacje zakończyły się sukcesem, commit wykonany.")

except Exception as e:
    # W przypadku błędu, wycofujemy wszystkie zmiany
    connection.rollback()
    print(f"Wystąpił błąd: {e}. Wykonano rollback.")

finally:
    # Zamykamy kursor i połączenie w tym miejscu, aby w przypadku wystąpienia błędu połączenie nie zostało otwarte
    cursor.close()
    connection.close()

Wszystkie operacje zakończyły się sukcesem, commit wykonany.


 ### Sprawdzenie
 Uruchamiamy poniższy kod, aby sprawdzić, czy faktycznie ta baza danych została zainicjowana prawidłowo.

In [121]:
# Używamy zmiennych środowiskowych do połączenia się z nowo utworzoną bazą danych
connection = psycopg2.connect(
    dbname=os.getenv("N_DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

cursor = connection.cursor()

In [123]:
def check_if_table_exists(table_name):
    msg = f"Sprawdzam czy istnieje tabela {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    # jeżeli tabela nie istnieje, ten krok zwróci wyjątek
    cursor.execute(query)
    print('OK!')

In [125]:
tables_to_test = [
    'aircraft',
    'airport_weather',
    'flight',
    'airport_list'
]

In [127]:
for table in tables_to_test:
    check_if_table_exists(table)

Sprawdzam czy istnieje tabela aircraft
OK!
Sprawdzam czy istnieje tabela airport_weather
OK!
Sprawdzam czy istnieje tabela flight
OK!
Sprawdzam czy istnieje tabela airport_list
OK!


In [129]:
connection.close()
msg = "Wszystko wygląda OK :) Przechodzimy do kolejnego zadania."
print(msg)

Wszystko wygląda OK :) Przechodzimy do kolejnego zadania.
